In [2]:
from dash import Dash,html,dcc
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input,Output,State
import dash_bootstrap_components as dbc

# insta_general

In [3]:
df = pd.read_csv('insta_general.csv', header=0)

In [4]:
df['posts']= df['posts'].astype('string').str.replace(',','').astype('int')

In [5]:
df['followers']= df['followers'].astype('int')
df['followings']= df['followings'].astype('int')

In [6]:
df0 = df.sort_values(by=['followers'])
followers_Bar = px.bar(df0 , y= 'name', x='followers', orientation='h')

In [7]:
df0 = df.sort_values(by=['posts'])
postsNum_Bar = px.bar(df0 , x= 'name', y='posts')

In [29]:
categories_followers_pie = px.pie(df, names='category', values='followers')

In [30]:
tier_pie = px.pie(df, names='tier')

In [10]:
df0 = df.sort_values(by=['avg_engagement_rate'])
er_Bar = px.bar(df0 , x= 'username', y='avg_engagement_rate')

# insta_detailed

In [11]:
df_detailed = pd.read_csv('insta_detailed.csv', header=0)

In [12]:
df_temp = df_detailed.drop(df_detailed[df_detailed['likes']=='unknown'].index)
df_detailed['likes'] = df_detailed['likes'].replace('unknown',df_temp['likes'].astype('int').min())
df_detailed['likes'] = df_detailed['likes'].astype('int')
df_detailed['captions']=df_detailed['captions'].fillna(' ')

In [31]:
posts_by_type_pie = px.pie(df_detailed, names='type')

In [14]:
df_detailed['datetimes'] = pd.to_datetime(df_detailed['datetimes'])

In [15]:
Likes_scatter = px.scatter(df_detailed,
                           x='datetimes',y='likes',
                           color='username',
                           hover_name='username',
                           width=1000,height=1000,
                           title='Date and Time of Post VS Likes',
                           symbol='username',
                           template='seaborn')

# Dash App

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output(component_id='graph1',component_property='figure'),
    Output(component_id='graph2',component_property='figure'),
    Output(component_id='graph3',component_property='figure'),
    Input(component_id='ddown1',component_property='value'),
    Input(component_id='ddown2',component_property='value'),
)
def update_My_Div(category,tier):
    if category==None and tier==None:
        df0 = df.sort_values(by=['followers'] , ascending=False)
        df1 = df.sort_values(by=['posts'], ascending=False)
        df2 = df.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar
    elif tier==None:
        df0 = df[df['category'] == category]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar
    elif category==None:
        df0 = df[df['tier'] == tier]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar,er_Bar
        
    else:
        df0 = df[np.logical_and(df['category'] == category,df['tier'] == tier)]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar
    
@app.callback(
    Output(component_id='graph4',component_property='figure'),
    Input(component_id='ddown3',component_property='value'),
)
def update_My_Div(username):
    if username==None or len(username)==0:
        Likes_scatter = px.scatter(df_detailed,
                        x='datetimes',y='likes',
                        color='username',
                        hover_name='username',
                        width=1000,height=1000,
                        title='Date_and_Time_of_Post VS Likes',
                        symbol='username',
                        template='seaborn')
        return Likes_scatter
    else:
        df_scatter = df_detailed[np.isin(df_detailed.username,username)]
        Likes_scatter = px.scatter(df_scatter,
                        x='datetimes',y='likes',
                        color='username',
                        hover_name='username',
                        width=1000,height=1000,
                        title='Date_and_Time_of_Post VS Likes',
                        symbol='username',
                        template='seaborn')
        return Likes_scatter

@app.callback(
    Output(component_id='ddown5',component_property='options'),
    Input(component_id='ddown4',component_property='value'),
)

def update_My_Div(username):
    df_name = df_detailed[df_detailed['username']==username]
    options=[{'label': str(datetimes), 'value': str(datetimes)}  for datetimes in df_name['datetimes'].unique()]

    return options

@app.callback(
    Output(component_id='card1',component_property='children'),
    Input(component_id='ddown4',component_property='value'),
    Input(component_id='ddown5',component_property='value'),
)

def update_My_Div(username, datetimes):
    
    df_name = df_detailed[df_detailed['username']==username]
    
    if username == None:
        children = [dbc.CardImg(src="https://raw.githubusercontent.com/ayahhhany/Instagram-Dashboard/main/avatars/insta.png", top=True),]
    elif datetimes == None or ((df_name['datetimes']!=datetimes).all()) :
        children = [dbc.CardImg(src = df[df['username'] == username].iloc[0,8], top=True),
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                   ]
    else:
        children = [dbc.CardImg(src = df[df['username'] == username].iloc[0,8], top=True),
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                    html.P(df_name[df_name['datetimes'] == datetimes].iloc[0,5], className="card-text",),
                    dbc.Button("Open Post", color="primary", href=df_name[df_name['datetimes'] == datetimes].iloc[0,1],),
                   ]

    return children
    
    

card_content1 = [ dbc.CardHeader("Combined Followers"),
                 dbc.CardBody([html.H5(str(df['followers'].sum()), className="card-title")]),]
card_content2 = [ dbc.CardHeader("Total Posts"),
                 dbc.CardBody([html.H5(str(df['posts'].sum()), className="card-title")]),]
card_content3 = [ dbc.CardHeader("Total Likes"),
                 dbc.CardBody([html.H5(str(df_detailed['likes'].sum()), className="card-title")]),]
card_content4 = [ dbc.CardHeader("Highest Engagement Rate"),
                 dbc.CardBody([html.H5(str(round(df['avg_engagement_rate'].max(),2)), className="card-title")]),]


card_content5 = [ dbc.CardHeader("Most Followed Category"),
                 dbc.CardBody([dcc.Graph(figure=categories_followers_pie)]),]
card_content6 = [ dbc.CardHeader("Tiers"),
                 dbc.CardBody([dcc.Graph(figure=tier_pie)]),]
card_content7 = [ dbc.CardHeader("Types of Posts"),
                 dbc.CardBody([dcc.Graph(figure=posts_by_type_pie)]),]

card_content8 = [ dbc.CardHeader("Date and Time of Post VS Likes"),
                 dbc.CardBody([dcc.Graph(id='graph4'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df_detailed['username'].unique()],id='ddown3',multi=True,),]),]

card_content9 = [ dbc.CardHeader("Discover Posts"),
                 dbc.CardBody([dbc.Card(id='card1'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df['username'].unique()],id='ddown4',),
     dcc.Dropdown(id='ddown5',),]),]

card_content10 = [ dbc.CardHeader("Who is most followed?"),
                   dbc.CardBody([html.Div(dcc.Graph(id='graph1')),]),]
card_content11 = [ dbc.CardHeader("Who posts the most?"),
                   dbc.CardBody([html.Div(dcc.Graph(id='graph2')),]),]
card_content12 = [ dbc.CardHeader("Who has the highest Engagement Rate?"),
                   dbc.CardBody([html.Div(dcc.Graph(id='graph3')),]),]

card_content13 = [ dbc.CardBody([
    dcc.Dropdown(options=[{'label': str(category), 'value': str(category)} \
                          for category in df['category'].unique()],
                  id='ddown1',
    ),
     dcc.Dropdown(options=[{'label': str(tier), 'value': str(tier)}  for tier in df['tier'].unique()],
                  id='ddown2',),]),]



app.layout =html.Div(children=[
    
 html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content1, color="primary", inverse=True)),
                dbc.Col(dbc.Card(card_content2, color="secondary", inverse=True)),
                dbc.Col(dbc.Card(card_content3, color="info", inverse=True)),
                dbc.Col(dbc.Card(card_content4, color="info", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content5, color="success", inverse=True)),
                dbc.Col(dbc.Card(card_content6, color="warning", inverse=True)),
                dbc.Col(dbc.Card(card_content7, color="danger", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content8, color="light")),
                dbc.Col(dbc.Card(card_content9, color="dark")),
            ], 
            className="mb-4"
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content10, color="success", inverse=True)),
                dbc.Col(dbc.Card(card_content11, color="warning", inverse=True)),
                dbc.Col(dbc.Card(card_content12, color="danger", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content13, color="success")),
            ],
            className="mb-4",
        ),
    ]
)
    ])
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_favicon.ico?v=2.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 11:21:54] "POST /_dash-update-component HTTP/1.1" 